<a href="https://colab.research.google.com/github/Payamfarahmand/Online_Retail_Prediction/blob/main/Online_Retail_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
# Importing the necessary libraries
import pandas as pd
import numpy as np
import seaborn as sns

# Define the file path
file_path = '/content/sample_data/project1/online_retail.csv'

# Load the CSV file
data = pd.read_csv(file_path)

# Display general information
print(data.info())
print(data.describe())
print(data.head())





<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB
None
            Quantity      UnitPrice     CustomerID
count  541909.000000  541909.000000  406829.000000
mean        9.552250       4.611114   15287.690570
std       218.081158      96.759853    1713.600303
min    -80995.000000  -11062.060000   12346.000000
25%         1.000000       1.250000   13953.000000
50%         3.000000       2.080000   15152.000000
75%        10.000000       4.130000

In [10]:
# Drop duplicate rows
data.drop_duplicates(inplace=True)

In [11]:
# Check for missing values
print(data.isnull().sum())

# Drop rows with missing values in key columns (e.g., CustomerID)
data.dropna(subset=['CustomerID'], inplace=True)

# Fill remaining missing values, if applicable (e.g., fill with median for numerical columns)
data['Quantity'].fillna(data['Quantity'].median(), inplace=True)
data['UnitPrice'].fillna(data['UnitPrice'].median(), inplace=True)


InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135037
Country             0
dtype: int64


<ipython-input-11-9e148571bea8>:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Quantity'].fillna(data['Quantity'].median(), inplace=True)
<ipython-input-11-9e148571bea8>:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=

In [12]:
# Remove rows where Quantity or UnitPrice is non-positive
data = data[(data['Quantity'] > 0) & (data['UnitPrice'] > 0)]


In [13]:
# Convert InvoiceDate to datetime
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'], errors='coerce')


In [14]:
# Add a TotalPrice column
data['TotalPrice'] = data['Quantity'] * data['UnitPrice']

# Extract PurchaseMonth for seasonal analysis
data['PurchaseMonth'] = data['InvoiceDate'].dt.month


In [15]:
# Remove extreme outliers using the interquartile range (IQR) method
Q1 = data['TotalPrice'].quantile(0.25)
Q3 = data['TotalPrice'].quantile(0.75)
IQR = Q3 - Q1
data = data[(data['TotalPrice'] >= (Q1 - 1.5 * IQR)) & (data['TotalPrice'] <= (Q3 + 1.5 * IQR))]


In [16]:
# Final check
print(data.info())
print(data.head())


<class 'pandas.core.frame.DataFrame'>
Index: 361461 entries, 0 to 541908
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   InvoiceNo      361461 non-null  object        
 1   StockCode      361461 non-null  object        
 2   Description    361461 non-null  object        
 3   Quantity       361461 non-null  int64         
 4   InvoiceDate    361461 non-null  datetime64[ns]
 5   UnitPrice      361461 non-null  float64       
 6   CustomerID     361461 non-null  float64       
 7   Country        361461 non-null  object        
 8   TotalPrice     361461 non-null  float64       
 9   PurchaseMonth  361461 non-null  int32         
dtypes: datetime64[ns](1), float64(3), int32(1), int64(1), object(4)
memory usage: 29.0+ MB
None
  InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053        

In [17]:
# Ensure CustomerID is treated as an integer type (if no decimals are expected)
data['CustomerID'] = data['CustomerID'].astype('Int64')

# Ensure InvoiceNo and StockCode are strings, as they are typically identifiers
data['InvoiceNo'] = data['InvoiceNo'].astype(str)
data['StockCode'] = data['StockCode'].astype(str)

# Recalculate TotalPrice to ensure it is accurate
data['TotalPrice'] = data['Quantity'] * data['UnitPrice']

# Check for any rows with non-positive Quantity or UnitPrice, then filter them out
data = data[(data['Quantity'] > 0) & (data['UnitPrice'] > 0)]

# Display updated information
print(data.info())
print(data.head())


<class 'pandas.core.frame.DataFrame'>
Index: 361461 entries, 0 to 541908
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   InvoiceNo      361461 non-null  object        
 1   StockCode      361461 non-null  object        
 2   Description    361461 non-null  object        
 3   Quantity       361461 non-null  int64         
 4   InvoiceDate    361461 non-null  datetime64[ns]
 5   UnitPrice      361461 non-null  float64       
 6   CustomerID     361461 non-null  Int64         
 7   Country        361461 non-null  object        
 8   TotalPrice     361461 non-null  float64       
 9   PurchaseMonth  361461 non-null  int32         
dtypes: Int64(1), datetime64[ns](1), float64(2), int32(1), int64(1), object(4)
memory usage: 29.3+ MB
None
  InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     710

In [18]:
# Define the file path for the output CSV file
output_path = '/content/sample_data/project1/cleaned_online_retail.csv'

# Export the cleaned data to a CSV file
data.to_csv(output_path, index=False)

print(f"Cleaned data exported to {output_path}")


Cleaned data exported to /content/sample_data/project1/cleaned_online_retail.csv
